# 音楽データセットの分類 (3)

- MFCC: メル周波数ケプストラム係数
  - 音声認識で用いられることが多い方法
  
### ソフトウェアのインストール

librosa のインストール<br>
- https://librosa.org
- https://anaconda.org/conda-forge/librosa

1. pythonがconda-forgeに置き換えられてしまっても構わなければ、普通にインストールも可能

```
conda install -c conda-forge librosa 
```

2. 仮想環境を作成してインストール

```
conda create -n librosa -c conda-forge python=3.9 scikit-learn scipy numpy pandas seaborn matplotlib jupyterlab librosa
```

- 仮想環境への移動<br>
```
conda activate librosa
```

- 仮想環境の終了<br>
```
conda deactivate
```

- 仮想環境の削除<br>
```
conda env remove -n librosa
```

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import librosa

# ジャンル名のリスト
genre_list = ['classical', 'jazz', 'pop', 'rock']
path = 'data/genres/'
nfiles = 100

_X = []
_Y = []

# 各ジャンルごとに100曲読み込み
for g in genre_list:
    for i in range(0, nfiles):
        file = '{}{}/{}.{:05}.wav'.format(path, g, g, i)
        data, sampling_rate = librosa.load(file, mono=True)
        mfcc = librosa.feature.mfcc(y=data, sr=sampling_rate)
        # 各次元の平均値を用いる
        _X.append(mfcc.mean(axis=1))
        _Y.append(genre_list.index(g))

# NumPyの配列に変換
X = np.array(_X)
Y = np.array(_Y)

# 教師データとテストデータに分割
# - データのサイズが大きく処理時間がかかるので、ここではホールドアウト法を用いる
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

分類器をテストする関数の定義

- 正確度の表示
- 混同行列の可視化

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

def test_model(model):
    # 教師データを学習
    model.fit(X_train, Y_train)
    # テストデータについてモデルからの予測値を計算
    Y_predict = model.predict(X_test)
    # 結果の表示
    print(classification_report(Y_test, Y_predict, target_names=genre_list, digits=3))
    # 混同行列の可視化
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(Y_test, Y_predict),
                xticklabels=genre_list, yticklabels=genre_list, 
                square=True, cbar=True, annot=True, cmap='Blues')
    plt.xlabel('Predicted Class')
    plt.ylabel('True Class')
    plt.show()

いくつかの分類器での精度の検証

- ナイーブベイズ
- ランダムフォレスト

In [ ]:
%%time
from sklearn.naive_bayes import GaussianNB

# ナイーブベイズ
model = GaussianNB()
# 分類器の精度検証
test_model(model)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

# ランダムフォレスト
model = RandomForestClassifier(n_jobs=-1)
# 分類器の精度検証
test_model(model)